In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
##parameters
radius = 25
p = 0.1
theta = 0.9
gamma = 0.8
grid_size = 100
center1 = 49
center2 = 49

In [ ]:
def writeto_csv(data,title):
    c1 = []
    c2 = []
    c3 = []
    for i in range(grid_size):
        for j in range(grid_size):
            c1.append(i+1)
            c2.append(j+1)
            c3.append(data[i][j])
    df = pd.DataFrame(list(zip(c1, c2, c3)), columns = ['index1','index2','value'])
    df.to_csv(title+ '.csv',index = False)
def read_csv(file):
    df = pd.read_csv(file)
    data = np.zeros(shape=(grid_size,grid_size))
    for index,row in df.iterrows():
        data[int(row['index1'])-1][int(row['index2'])-1] = float(row['value'])
    return data

In [ ]:
##input construction and noise addition
X = np.zeros(shape = (grid_size,grid_size))            
for i in range(grid_size):
    for j in range(grid_size):
        if (i-center1)**2 + (j-center2)**2 <= radius**2:
            X[i][j] = 1
            
# Y = np.zeros(shape = (grid_size,grid_size))

# np.random.seed(0)
# tmp = np.random.random_sample((grid_size,grid_size))
# for i in range(grid_size):
#     for j in range(grid_size):
        
#         if tmp[i][j]<p:
#             if X[i][j] == 1:
#                 Y[i][j] = 0
#             else:
#                 Y[i][j] = 1
#         else:
#             Y[i][j] = X[i][j]
            
#csv file should have three columns as index1, index2, value
Y = read_csv('input.csv')

In [ ]:
def make_graph(nodes,edges,adj):
        edges.append((nodes[0],nodes[1]))
        try:
            t = adj[nodes[0]]
            adj[nodes[0]].append(nodes[1])
        except:
            adj[nodes[0]] = [nodes[1]]

        edges.append((nodes[1],nodes[0]))
        try:
            t = adj[nodes[1]]
            adj[nodes[1]].append(nodes[0])
        except:
            adj[nodes[1]] = [nodes[0]]
            
        edges.append((nodes[1],nodes[2]))
        try:
            t = adj[nodes[1]]
            adj[nodes[1]].append(nodes[2])
        except:
            adj[nodes[1]] = [nodes[2]]

        edges.append((nodes[2],nodes[1]))
        try:
            t = adj[nodes[2]]
            adj[nodes[2]].append(nodes[1])
        except:
            adj[nodes[2]] = [nodes[1]]

In [ ]:
edges = []
adj = {}
for i in range(grid_size):
    for j in range(grid_size):
        node1 = i*grid_size+j
        node2 = i*grid_size+j+1
        node3 = (i+1)*grid_size+j
        node4 = (grid_size)*(grid_size) + node1
        if j==grid_size-1:
            node2 = -1
        if i==grid_size-1:
            node3 = -1

        make_graph([(node1,),(node1,node4),(node4,)] ,edges, adj)
        if node2!=-1:
            make_graph([(node1,), (node1,node2), (node2,)] ,edges, adj)
        if node3!=-1:
            make_graph([(node1,), (node1,node3), (node3,)] ,edges, adj)
            
msgs = {}
for i in range(len(edges)):
    msgs[edges[i]] = [1,1]

In [ ]:
def msg_update(edge):
    node1 = edge[0]
    node2 = edge[1]
    if len(node1) == 2:
        ##case 1, 3,5
        if node2[0] >= grid_size*grid_size:
            #case 5 no need to handle
            pass
        else:
            #case 1,3
            tmp = -1
            if node1[0] == node2[0]:
                tmp = node1[1]
            else:
                tmp = node1[0]
            if tmp >= grid_size*grid_size:
                #case 3
                tmp -= grid_size*grid_size
                ind1 = int(tmp/grid_size)
                ind2 = int(tmp%grid_size)
                y = Y[ind1][ind2]
                if y == 1:
                    req_msg = [(1-gamma) , (1+gamma)]
                    req_msg = req_msg/np.sum(req_msg)
                    msgs[edge] = req_msg.copy()
                else:
                    req_msg = [(1+gamma), (1-gamma)]
                    req_msg = req_msg/np.sum(req_msg)
                    msgs[edge] = req_msg.copy()
            else:
                #case 1
                neighs = adj[node1].copy()
                neighs.remove(node2)
                neigh_msgs = [1,1]
                for neigh in neighs:
                    msg = msgs[(neigh,node1)]
                    neigh_msgs[0] *= msg[0]
                    neigh_msgs[1] *= msg[1]
                req_msg = [1,1]
                req_msg[0] = max(neigh_msgs[0]*(1+theta) , neigh_msgs[1]*(1-theta))
                req_msg[1] = max(neigh_msgs[0]*(1-theta) , neigh_msgs[1]*(1+theta))
                req_msg = req_msg/np.sum(req_msg)
                msgs[edge] = req_msg.copy()
            
    else:
        if node1[0] >=grid_size*grid_size:
            pass
            #case 6
            ##no need to handle
        else:  
            #case 2 4
            neighs = adj[node1].copy()
            neighs.remove(node2)
            neigh_msgs = [1,1]
            for neigh in neighs:
                msg = msgs[(neigh,node1)]
                neigh_msgs[0] *= msg[0]
                neigh_msgs[1] *= msg[1]
            neigh_msgs = neigh_msgs/np.sum(neigh_msgs)
            msgs[edge] = neigh_msgs.copy()

In [ ]:
iters = 20
for i in tqdm(range(iters)):
    np.random.shuffle(edges)
    for j in range(len(edges)):
        msg_update(edges[j])

In [ ]:
X1 = np.zeros(shape=(grid_size,grid_size))
for i in range(grid_size):
    for j in range(grid_size):
        node = (i*grid_size+j,)
        neighs = adj[node].copy()
        neigh_msgs = [1,1]
        for neigh in neighs:
            msg = msgs[(neigh,node)]
            neigh_msgs[0] *= msg[0]
            neigh_msgs[1] *= msg[1]
        neigh_msgs = neigh_msgs/np.sum(neigh_msgs)
        X1[i][j] = np.argmax(neigh_msgs)
print(len(np.where(X1 == X)[0]))

In [ ]:
##input plot
plt.imshow(Y, cmap=plt.cm.gray)
plt.title('Noisy image for p = '+str(p))
plt.show()

In [ ]:
##output plot
plt.imshow(X1, cmap=plt.cm.gray)
plt.title('Denoised image for p = '+str(p))
plt.show()

In [ ]:
writeto_csv(X1,'output')

In [ ]:
##for calculating averages 4th part takes time
runs = 10
radius = 25
p = 0.2
grid_size = 100
center1 = 49
center2 = 49
iters = 20
theta_gamma_pairs = [(0.9,0.9),(0.9,0.8),(0.9,0.7),(0.7,0.9),(0.2,0.1)]
for i in range(len(theta_gamma_pairs)):
    theta = theta_gamma_pairs[i][0]
    gamma = theta_gamma_pairs[i][1]
    count = 0
    def make_graph(nodes,edges,adj):
            edges.append((nodes[0],nodes[1]))
            try:
                t = adj[nodes[0]]
                adj[nodes[0]].append(nodes[1])
            except:
                adj[nodes[0]] = [nodes[1]]

            edges.append((nodes[1],nodes[0]))
            try:
                t = adj[nodes[1]]
                adj[nodes[1]].append(nodes[0])
            except:
                adj[nodes[1]] = [nodes[0]]
                
            edges.append((nodes[1],nodes[2]))
            try:
                t = adj[nodes[1]]
                adj[nodes[1]].append(nodes[2])
            except:
                adj[nodes[1]] = [nodes[2]]

            edges.append((nodes[2],nodes[1]))
            try:
                t = adj[nodes[2]]
                adj[nodes[2]].append(nodes[1])
            except:
                adj[nodes[2]] = [nodes[1]]
                
    edges = []
    adj = {}
    for i in range(grid_size):
        for j in range(grid_size):
            node1 = i*grid_size+j
            node2 = i*grid_size+j+1
            node3 = (i+1)*grid_size+j
            node4 = (grid_size)*(grid_size) + node1
            if j==grid_size-1:
                node2 = -1
            if i==grid_size-1:
                node3 = -1

            make_graph([(node1,),(node1,node4),(node4,)] ,edges, adj)
            if node2!=-1:
                make_graph([(node1,), (node1,node2), (node2,)] ,edges, adj)
            if node3!=-1:
                make_graph([(node1,), (node1,node3), (node3,)] ,edges, adj)

    X = np.zeros(shape = (grid_size,grid_size))            
    for i in range(grid_size):
        for j in range(grid_size):
            if (i-center1)**2 + (j-center2)**2 <= radius**2:
                X[i][j] = 1
                
    for i in tqdm(range(runs)):
        Y = np.zeros(shape = (grid_size,grid_size))

        for i in range(grid_size):
            for j in range(grid_size):
                tmp = np.random.random()
                if tmp<p:
                    if X[i][j] == 1:
                        Y[i][j] = 0
                    else:
                        Y[i][j] = 1
                else:
                    Y[i][j] = X[i][j]
        msgs = {}
        for i in range(len(edges)):
            msgs[edges[i]] = [1,1]
            
        def msg_update(edge):
            node1 = edge[0]
            node2 = edge[1]
            if len(node1) == 2:
                ##case 1, 3,5
                if node2[0] >= grid_size*grid_size:
                    #case 5 no need to handle
                    pass
                else:
                    #case 1,3
                    tmp = -1
                    if node1[0] == node2[0]:
                        tmp = node1[1]
                    else:
                        tmp = node1[0]
                    if tmp >= grid_size*grid_size:
                        #case 3
                        tmp -= grid_size*grid_size
                        ind1 = int(tmp/grid_size)
                        ind2 = int(tmp%grid_size)
                        y = Y[ind1][ind2]
                        if y == 1:
                            req_msg = [(1-gamma) , (1+gamma)]
                            req_msg = req_msg/np.sum(req_msg)
                            msgs[edge] = req_msg.copy()
                        else:
                            req_msg = [(1+gamma), (1-gamma)]
                            req_msg = req_msg/np.sum(req_msg)
                            msgs[edge] = req_msg.copy()
                    else:
                        #case 1
                        neighs = adj[node1].copy()
                        neighs.remove(node2)
                        neigh_msgs = [1,1]
                        for neigh in neighs:
                            msg = msgs[(neigh,node1)]
                            neigh_msgs[0] *= msg[0]
                            neigh_msgs[1] *= msg[1]
                        req_msg = [1,1]
                        req_msg[0] = max(neigh_msgs[0]*(1+theta) , neigh_msgs[1]*(1-theta))
                        req_msg[1] = max(neigh_msgs[0]*(1-theta) , neigh_msgs[1]*(1+theta))
                        req_msUntitled0g = req_msg/np.sum(req_msg)
                        msgs[edge] = req_msg.copy()

            else:
                if node1[0] >=grid_size*grid_size:
                    pass
                    #case 6
                    ##no need to handle
                else:  
                    #case 2 4
                    neighs = adj[node1].copy()
                    neighs.remove(node2)
                    neigh_msgs = [1,1]
                    for neigh in neighs:
                        msg = msgs[(neigh,node1)]
                        neigh_msgs[0] *= msg[0]
                        neigh_msgs[1] *= msg[1]
                    neigh_msgs = neigh_msgs/np.sum(neigh_msgs)
                    msgs[edge] = neigh_msgs.copy()
                    
        for i in range(iters):
            np.random.shuffle(edges)
            for j in range(len(edges)):
                msg_update(edges[j])
        
        X1 = np.zeros(shape=(grid_size,grid_size))
        for i in range(grid_size):
            for j in range(grid_size):
                node = (i*grid_size+j,)
                neighs = adj[node].copy()
                neigh_msgs = [1,1]
                for neigh in neighs:
                    msg = msgs[(neigh,node)]
                    neigh_msgs[0] *= msg[0]
                    neigh_msgs[1] *= msg[1]
                neigh_msgs = neigh_msgs/np.sum(neigh_msgs)
                X1[i][j] = np.argmax(neigh_msgs)
        count += len(np.where(X1 == X)[0])
    print(count/runs)